# Vacation Py

In [159]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [160]:
cities_weather_df = pd.read_csv('output data/cities.csv')
cities_weather_df = cities_weather_df.drop(columns= "Unnamed: 0")
cities_weather_df



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saldanha,-33.0117,17.9442,60.85,90,12,20.47,ZA,1666558061
1,George Town,5.4112,100.3354,80.53,86,40,14.00,MY,1666557782
2,Barrow,71.2906,-156.7887,21.22,86,20,20.71,US,1666558107
3,Saskylakh,71.9167,114.0833,3.31,99,67,5.77,RU,1666558059
4,Hilo,19.7297,-155.0900,80.33,71,100,6.91,US,1666558075
...,...,...,...,...,...,...,...,...,...
533,Faya,18.3851,42.4509,62.71,66,17,2.86,SA,1666558078
534,Koygorodok,60.4450,50.9968,36.19,92,100,13.65,RU,1666558603
535,Cloquet,46.7216,-92.4594,74.35,64,0,13.80,US,1666558603
536,Mzimba,-11.9000,33.6000,72.28,43,74,12.86,MW,1666558603


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [161]:
gmaps.configure(api_key=g_key)

In [162]:
locations = cities_weather_df[['Lat', 'Lng']]
humidity = cities_weather_df["Humidity"]
max_humidity = float(humidity.max())
layout={'width': '600px',
        'height': '420px',
        'padding': '3px',
        'border': '1px solid black',
        'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=layout, center = [0, 0], zoom_level=1.2)
heatmap = gmaps.heatmap_layer(locations, weights=humidity)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='3px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [163]:
ideal_weather_df = cities_weather_df.loc[(cities_weather_df["Max Temp"] > 70) &
                                         (cities_weather_df["Max Temp"] < 100) &
                                         (cities_weather_df["Wind Speed"] < 10) &
                                         (cities_weather_df["Cloudiness"] == 0)]
                                    
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,Fomboni,-12.2800,43.7425,77.32,78,0,9.28,KM,1666558226
86,Jamestown,42.0970,-79.2353,71.24,32,0,3.44,US,1666557826
124,Cookeville,36.1628,-85.5016,76.71,36,0,8.05,US,1666558404
188,Rio Grande,-32.0350,-52.0986,71.71,82,0,7.78,BR,1666558072
274,Sur,22.5667,59.5289,80.08,39,0,8.43,OM,1666558531
315,Ratnagiri,16.9833,73.3000,72.84,63,0,7.31,IN,1666558541
322,Cayenne,4.9333,-52.3333,84.24,74,0,8.05,GF,1666558250
349,Abu Dhabi,24.4667,54.3667,84.22,72,0,3.44,AE,1666558534
390,Jardim,-21.4803,-56.1381,80.80,59,0,4.12,BR,1666558560
398,Verāval,20.9000,70.3667,78.85,47,0,4.56,IN,1666558563


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [164]:
hotel_name = []

In [165]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index, row in ideal_weather_df.iterrows():
    
    target_coordinates = f"{row['Lat']}, {row['Lng']}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
        }

    response = requests.get(base_url, params=params)
    places_data = response.json()
    hotel_name.append(places_data['results'][0]['name'])


    


In [166]:
hotel_df = pd.DataFrame({"Hotel Name": hotel_name,
                         "City": ideal_weather_df["City"],
                         "Lat": ideal_weather_df["Lat"],
                         "Lng": ideal_weather_df["Lng"],
                         "Country": ideal_weather_df["Country"],
                        })
hotel_df


,Hotel Name,City,Lat,Lng,Country
24,Le Relais de Singani hôtel,Fomboni,-12.2800,43.7425,KM
86,Chautauqua Harbor Hotel,Jamestown,42.0970,-79.2353,US
124,"Holiday Inn Cookeville, an IHG Hotel",Cookeville,36.1628,-85.5016,US
188,Hotel Laghetto Viverone Rio Grande,Rio Grande,-32.0350,-52.0986,BR
274,Sur Grand Hotel,Sur,22.5667,59.5289,OM
315,Windsongs A Seaview Premium Homestay,Ratnagiri,16.9833,73.3000,IN
322,Grand Hotel Montabo,Cayenne,4.9333,-52.3333,GF
349,Four Seasons Hotel Abu Dhabi at Al Maryah Island,Abu Dhabi,24.4667,54.3667,AE
390,Hotel Jardim,Jardim,-21.4803,-56.1381,BR
398,The Fern Residency Somnath,Verāval,20.9000,70.3667,IN


In [167]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [168]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='3px', wi…